# SKEMA-TA2-UAZ Incremental Structral Alignment Demo (2023-01-23)

**Authors**: Liang Zhang, Adarsh Pyarelal, Clayton Morrison

Pipeline: PDF document -> Equation images -> MathML representation -> *Graph representation -> *Equation alignment

\* What we implement in this demo

The overall goals of this demo are:
- Convert MathML representation to graph represention
- Align two equations and return the matching ratio and the varible alignment result

Swagger docs for the REST API can be found at http://localhost:8080/docs/

## Incremental structural alignment

**A quick review**: We proposed using seeded graph matching (SGM) to achieve incremental structural alignment (ISA) of equations. At a high level, the procedure is as follows:

1. Create a graph representation based on the MathML input.
2. Construct the adjacency matrices corresponding to the above graph representations. 
3. Apply the SGM algorithm with the two adjacency matrices as inputs.


* Please apply 'pip install requests pydot graspologic' if you don't have them installed in your machine.

We will align the core equations from SEIR and SEIRD+V as an example.

In [1]:
from IPython.display import Image

In [2]:
Image(url="data/seir_core.png", width=200, height=200) # Human in the loop: identify the core of SEIR

In [3]:
Image(url="data/seirdv_core.png", width=200, height=200) # Human in the loop: identify the core of SEIRD+V

In [4]:
# -*- coding: utf-8 -*
import warnings
warnings.filterwarnings('ignore')
import requests
import pydot
import numpy as np
from graspologic.match import graph_match
from graphviz import Source

In [5]:
np.random.seed(1)
rng = np.random.default_rng(1)

Operator encodings in adjacency matrix

In [6]:
op_dict = {"+": 1, "-": 2, "*": 3, "/": 4, "=": 5, "√": 6}

Call the graph generation API

In [7]:
def generate_graph(file="seir_eq1.xml", render=False):
    if '<math>' in file and '</math>' in file:
        content = file
    else:
        with open("data/" + file) as f:
            content = f.read()

    digraph = requests.put('http://localhost:8080/mathml/math-exp-graph', data=content.encode('utf-8'))
    if render:
        src = Source(digraph.text)
        src.render('doctest-output/mathml_exp_tree', view=True)
    graph = pydot.graph_from_dot_data(str(digraph.text))[0]
    return graph

Generate adjacency matrix based on the graph representation

In [8]:
def generate_amatrix(graph):
    node_labels = []
    for node in graph.get_nodes():
        node_labels.append(node.obj_dict['attributes']['label'].replace('"', ''))

    amatrix = np.zeros((len(node_labels), len(node_labels)))

    for edge in graph.get_edges():
        x, y = edge.obj_dict['points']
        label = edge.obj_dict['attributes']['label'].replace('"', '')
        amatrix[int(x)][int(y)] = op_dict[label] if label in op_dict else 7

    return amatrix, node_labels

Align two equations

In [9]:
def align_mathml_eqs(file1, file2):
    graph1 = generate_graph(file1)
    graph2 = generate_graph(file2)

    amatrix1, node_labels1 = generate_amatrix(graph1)
    amatrix2, node_labels2 = generate_amatrix(graph2)

    seed1 = [0, 1]
    seed2 = [0, 1]
    for i in range(2, len(node_labels1)):
        for j in range(2, len(node_labels2)):
            if node_labels1[i].lower() == node_labels2[j].lower():
                seed1.append(i)
                seed2.append(j)

    partial_match = np.column_stack((seed1, seed2))

    matched_indices1, matched_indices2, _, _ = graph_match(
        amatrix1, amatrix2, partial_match=partial_match, padding="adopted", rng=rng, max_iter=50
    )

    big_graph_idx = 0 if len(node_labels1) >= len(node_labels2) else 1
    if big_graph_idx == 0:
        big_graph = amatrix1
        big_graph_matched_indices = matched_indices1
        small_graph = amatrix2
        small_graph_matched_indices = matched_indices2
    else:
        big_graph = amatrix2
        big_graph_matched_indices = matched_indices2
        small_graph = amatrix1
        small_graph_matched_indices = matched_indices1

    small_graph_aligned = small_graph[small_graph_matched_indices][:, small_graph_matched_indices]
    small_graph_aligned_full = np.zeros(big_graph.shape)
    small_graph_aligned_full[np.ix_(big_graph_matched_indices, big_graph_matched_indices)] = small_graph_aligned

    num_edges = ((big_graph + small_graph_aligned_full) > 0).sum()
    diff_edges = abs(big_graph - small_graph_aligned_full)
    diff_edges[diff_edges > 0] = 1
    num_diff_edges = np.sum(diff_edges)
    matching_ratio = round(1 - (num_diff_edges / num_edges), 2)

    long_len = len(node_labels1) if len(node_labels1) >= len(node_labels2) else len(node_labels2)
    aligned_indices1 = np.zeros((long_len)) - 1
    aligned_indices2 = np.zeros((long_len)) - 1
    for i in range(long_len):
        if i < len(node_labels1):
            if i in matched_indices1:
                aligned_indices1[i] = matched_indices2[np.where(matched_indices1 == i)[0][0]]
                aligned_indices2[matched_indices2[np.where(matched_indices1 == i)[0][0]]] = i

    return matching_ratio, node_labels1, node_labels2, aligned_indices1, aligned_indices2

Visualize the graph of the derivative of S in the SEIR paper

In [10]:
generate_graph("seir_eq1.xml", render=True)

Visualize the graph of the derivative of S in the SEIRD+V paper

In [11]:
generate_graph("seirdv_eq2.xml", render=True)

Print the alignment results for the derivatives of S in the SEIR paper and in SEIRD+V

In [12]:
matching_ratio, node_labels1, node_labels2, aligned_indices1, aligned_indices2 = align_mathml_eqs("seir_eq1.xml", "seirdv_eq2.xml")

print('matching ratio: ' + str(round(matching_ratio * 100, 2)) + '%')
for i in range(len(node_labels1)):
    if aligned_indices1[i] != -1:
        print(str(node_labels1[i]) + '<=====>' + str(node_labels2[int(aligned_indices1[i])]))
    else:
        print(str(node_labels1[i]) + '<=====>missing')

for i in range(len(node_labels2)):
    if i not in aligned_indices1:
        print('missing<=====>' + str(node_labels2[i]))

matching ratio: 100.0%
Λ-μ*S-β*S*I/N<=====>ı-μ*S-β/N*I*S
˙(S)<=====>derivative(s, t)
Λ<=====>ı
μ*S<=====>μ*S
μ<=====>μ
S<=====>S
β*S*I/N<=====>β/N*I*S
β<=====>β
I<=====>I
N<=====>N


Visualize the graph of the derivative of E in SEIRD+V

In [13]:
generate_graph("seirdv_eq3.xml", render=True)

Print the alignment results for the derivative of S in the SEIR paper and the derivative of E in SEIRD+V

In [14]:
matching_ratio, node_labels1, node_labels2, aligned_indices1, aligned_indices2 = align_mathml_eqs("seir_eq1.xml", "seirdv_eq3.xml")

print('matching ratio: ' + str(round(matching_ratio * 100, 2)) + '%')
for i in range(len(node_labels1)):
    if aligned_indices1[i] != -1:
        print(str(node_labels1[i]) + '<=====>' + str(node_labels2[int(aligned_indices1[i])]))
    else:
        print(str(node_labels1[i]) + '<=====>missing')

for i in range(len(node_labels2)):
    if i not in aligned_indices1:
        print('missing<=====>' + str(node_labels2[i]))

matching ratio: 36.0%
Λ-μ*S-β*S*I/N<=====>β/N*I*S-(μ+ε)*E
˙(S)<=====>derivative(E, t)
Λ<=====>E
μ*S<=====>μ+ε
μ<=====>μ
S<=====>S
β*S*I/N<=====>β/N*I*S
β<=====>β
I<=====>I
N<=====>N
missing<=====>(μ+ε)*E
missing<=====>ε


Therefore, based on the matching ratio, we can identify consistent equations in the two papers and align their variables, as well as identify several equations added in SEIRD+V that cannot be matched to any SEIR equations.

In [15]:
seir_eq_files = ['seir_eq1.xml', 'seir_eq2.xml', 'seir_eq3.xml', 'seir_eq4.xml']
seirdv_eq_files = ['seirdv_eq1.xml', 'seirdv_eq2.xml', 'seirdv_eq3.xml', 'seirdv_eq4.xml', 'seirdv_eq5.xml',
                   'seirdv_eq6.xml', 'seirdv_eq7.xml']

paper_1_missing_eqs = [i for i in range(len(seir_eq_files))]
paper_2_missing_eqs = [i for i in range(len(seirdv_eq_files))]
for i in range(len(seir_eq_files)):
    for j in range(len(seirdv_eq_files)):
        matching_ratio, _, _, _, _ = align_mathml_eqs(
            seir_eq_files[i], seirdv_eq_files[j])
        if matching_ratio > 0.9:
            print('Equation ' + str(i + 1) + ' <=====> Equation ' + str(j + 1))
            paper_1_missing_eqs.remove(i)
            paper_2_missing_eqs.remove(j)

for p1 in paper_1_missing_eqs:
    print('Equation ' + str(p1 + 1) + '<=====> missing')

for p2 in paper_2_missing_eqs:
    print('missing <=====> Equation ' + str(p2 + 1))

Equation 1 <=====> Equation 2
Equation 2 <=====> Equation 3
Equation 3 <=====> Equation 4
Equation 4 <=====> Equation 5
missing <=====> Equation 1
missing <=====> Equation 6
missing <=====> Equation 7
